# Segmenting and Clustering Neighborhoods in Toronto

### Assignment for the week 3 of the Coursera course"Applied Data Science Capstone"

Import requests package to get the wikipedia page and the BeautifulSoup module for scraping the web page

In [1]:
import requests
from bs4 import BeautifulSoup

Get the wikipedia page

In [2]:
link='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(link)

Transform the page in BeautifulSoup object for scraping

In [3]:
soup=BeautifulSoup(page.content,'html.parser')

In [17]:
#print(soup.prettify)

Get the table with the data

In [18]:
table=list(soup.find_all('table',class_='wikitable sortable'))[0]
#table

Get the list of the rows of the table

In [19]:
tr_list=table.find_all('tr')
tr_list.pop(0) # delete the firs element, containing the column labels
#tr_list

<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

Bulild a list of dictionaries with the data following the requirenments of the asseignment

In [12]:
data=[]
pcode_prev = ''
for i, row in enumerate(tr_list):                  # iterate on the list of rows
    td_list = row.find_all('td')                   # build a list of the cells in the row
    borough = td_list[1].get_text()                # extract the value of borough in string format
    neighborhood = td_list[2].get_text().rstrip()  # extract the value of neighborhood in string format removing the "\n" characters
    if neighborhood == "Not assigned":
        neighborhood = borough
    if i == 0:                                     # to initialize the variables neighborhoods and borough_prev
        neighborhoods = neighborhood
        borough_prev = borough
    if( borough != "Not assigned"):
        pcode = td_list[0].get_text()        # current value of postal code
        if i == 0:
            data.append({'PostalCode': pcode,'Borough': borough,'Neighborhood': neighborhoods})
        else:
            if( pcode == pcode_prev):        # I'm assuming rows with the same postal code are in successive order 
                neighborhoods = neighborhoods + ', ' + neighborhood   # concatenate the neighborhood names
            else:                            # when find different postal code then append the row with the values of the previous postal code
                data.append({'PostalCode': pcode_prev,'Borough': borough_prev,'Neighborhood': neighborhoods})
                neighborhoods = neighborhood
        pcode_prev = pcode                  # register the previous vaule of postal code
        borough_prev = borough

Import pandas package

In [13]:
import pandas as pd

Build the dataframe from the list of dictionaries just built.

In [16]:
df=pd.DataFrame(data)
df=df[['PostalCode','Borough','Neighborhood']]
df=df.drop(df.index[0])         # remove the first line
df.head()

,PostalCode,Borough,Neighborhood
1,M3A,North York,Parkwoods
2,M4A,North York,Victoria Village
3,M5A,Downtown Toronto,"Harbourfront, Regent Park"
4,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Queen's Park,Queen's Park


Dataframe shape:

In [11]:
df.shape

(102, 3)